# Neural nets to solve a physics problem


In [3]:
# http://pytorch.org/
# https://keras.io/
!pip install -q keras
from os import path
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras import optimizers
from sklearn.cross_validation import train_test_split
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install tqdm
import torch

import numpy as np
import torch
from torch.autograd import Variable
from torch import nn, optim
from tqdm import tqdm, trange
def V(x):   # converts a numpy array to a pyTorch variable
    return Variable(torch.FloatTensor(np.array(x)), requires_grad=True)
def C(x):   # same as above; but this is for y0 (the actual value), not y (the NN-output value); pyTorch actually forces me to do this...
    return Variable(torch.FloatTensor(np.array(x)), requires_grad=False)
def N(x):   # converts back
    return x.data.cpu().numpy()
 
Nsamples = 20000
max_Nfeatures = 100
epochs = 300
 
# generate data
Xs = []
ys = []
for isample in trange(Nsamples):
    # a random number of features
    Nfeatures = np.random.randint(max_Nfeatures-40) + 40
 
    # generate X
    # intermediateX example: shape [50, 3], content [[-25,25,44], [-10,30,40], ....]
    intermediateX = np.random.rand(Nfeatures, 3) * 50 - 50 * np.random.rand(3)
    # X example: shape [2500, 6], content [[-25,25,44,-25,25,44], [-25,25,44,-10,30,40], ...]
    X = np.zeros((Nfeatures, Nfeatures, 6))
    X[:,:,:3] = np.expand_dims(intermediateX, 1)
    X[:,:,3:] = np.expand_dims(intermediateX, 0)
    X = X.reshape(-1,6)
    X = X * np.mgrid[1:2:len(X)]   # a modification that would seem meaningless from your point of view... just keep it...
    # Note: all these is similar to below; but I found my data-generating method to generate much-harder-to-train data
    # X = np.random.rand(Nfeatures * Nfeatures, 6)
 
    # generate y
    origin = V([0,0,0])
    # firstly, z=f(x)
    X1 = V(X[:,:3]) - origin   # example shape: (2500, 3); origin=0 so ignore origin for now
    X2 = V(X[:,3:]) - origin
    R1 = torch.norm(X1, dim=1)       # example shape: (2500, 1)
    R2 = torch.norm(X2, dim=1)
    z = 1 / R1 / R2      # example shape: (2500, 1)
    z = torch.sum(z) * 1e4        # example shape: 1
    # then y=g(z). why not just ask you to train z? because training y is much more difficult than z.
    y = N(torch.autograd.grad(z, origin, create_graph=True)[0])
 
    Xs.append(np.mean(X, 0))
    ys.append(y / (len(X)))
     

print("\n")
print(Xs[0].shape)
print(y.shape)
print(y)
print(X[0])
print(Xs[0])
print(ys[0])


X_train, X_test, y_train, y_test = train_test_split(Xs, ys, train_size = 0.8)
print(type(X_train))

print("train")
print(len(X_train))
print(X_train[0].shape)
model = Sequential()
model.add(Dense(units=400, input_dim=6))
model.add(Activation('relu'))
model.add(Dense(units=200))
model.add(Activation('relu'))
model.add(Dense(units=100))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(units=50))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Activation('relu'))
model.add(Dense(units=3))

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error',
              optimizer='sgd',metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=50, verbose=2)

loss_and_metrics = model.evaluate(np.array(X_test), np.array(y_test), batch_size=100)

classes = model.predict(np.array(X_test), batch_size=1)

print(classes[:10])
print(y_test[:10])
"""
# neural net
net = nn.Sequential(
    nn.Linear(6, 192),
    nn.ReLU(),
    nn.Linear(192, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)
optimizer = optim.Adam(net.parameters(), lr=1E-4)
criterion = nn.MSELoss()
net.train()

# train
for epoch in trange(epochs * Nsamples): # using trange instead of range produces a progressbar
    # randomly select a sample
    isample = np.random.randint(0, Nsamples)
    y0 = C(ys[isample])
 
    # calculate neural net output
    X = V(Xs[isample]) - torch.cat([origin, origin])
    z = torch.sum(net(X))
    # z = torch.sum(1 / torch.norm(X[:,:3], dim=1) / torch.norm(X[:,3:], dim=1) * 1e4)  # this is 100% accurate
    y = torch.autograd.grad(z, origin, create_graph=True)[0]
 
    # and perform train step
    loss = criterion(y,y0)
    optimizer.zero_grad()   # suggested trick
    loss.backward()
    optimizer.step()
 
    if epoch % 1000 == 0:
        tqdm.write('%s\t%s\t%s\t%s' %(epoch, N(loss)[0], N(y0)[0], N(y)[0]))
"""


100%|██████████| 20000/20000 [00:39<00:00, 502.79it/s]




(6,)
(3,)
[-2906.2344   -915.64734 -1517.0038 ]
[-15.84931198  15.02764017 -10.42799311 -15.84931198  15.02764017
 -10.42799311]
[ 18.19428478   6.20820177 -11.25640949  18.19428478   6.20820177
 -11.25640949]
[ 0.69157505  0.30031165 -0.09098879]
<type 'list'>
train
16000
(6,)
Epoch 1/100
 - 1s - loss: 1.9295 - acc: 0.5561
Epoch 2/100
 - 1s - loss: 1.8847 - acc: 0.6228
Epoch 3/100
 - 1s - loss: 1.8742 - acc: 0.6557
Epoch 4/100
 - 1s - loss: 1.8682 - acc: 0.6699
Epoch 5/100
 - 1s - loss: 1.8658 - acc: 0.6744
Epoch 6/100
 - 1s - loss: 1.8635 - acc: 0.6800
Epoch 7/100
 - 1s - loss: 1.8624 - acc: 0.6855
Epoch 8/100
 - 1s - loss: 1.8624 - acc: 0.6888
Epoch 9/100
 - 1s - loss: 1.8603 - acc: 0.6919
Epoch 10/100
 - 1s - loss: 1.8583 - acc: 0.6903
Epoch 11/100
 - 1s - loss: 1.8612 - acc: 0.6939
Epoch 12/100
 - 1s - loss: 1.8552 - acc: 0.6963
Epoch 13/100
 - 1s - loss: 1.8568 - acc: 0.7053
Epoch 14/100
 - 1s - loss: 1.8577 - acc: 0.7022
Epoch 15/100
 - 1s - loss: 1.8527 - acc: 0.7085
Epoch 16

Epoch 41/100
 - 1s - loss: 1.8467 - acc: 0.7154
Epoch 42/100
 - 1s - loss: 1.8510 - acc: 0.7134
Epoch 43/100
 - 1s - loss: 1.8463 - acc: 0.7174
Epoch 44/100
 - 1s - loss: 1.8447 - acc: 0.7164
Epoch 45/100
 - 1s - loss: 1.8473 - acc: 0.7132
Epoch 46/100
 - 1s - loss: 1.8465 - acc: 0.7186
Epoch 47/100
 - 1s - loss: 1.8463 - acc: 0.7214
Epoch 48/100
 - 1s - loss: 1.8450 - acc: 0.7193
Epoch 49/100
 - 1s - loss: 1.8456 - acc: 0.7259
Epoch 50/100
 - 1s - loss: 1.8449 - acc: 0.7188
Epoch 51/100
 - 1s - loss: 1.8452 - acc: 0.7176
Epoch 52/100
 - 1s - loss: 1.8434 - acc: 0.7196
Epoch 53/100
 - 1s - loss: 1.8468 - acc: 0.7216
Epoch 54/100
 - 1s - loss: 1.8453 - acc: 0.7211
Epoch 55/100
 - 1s - loss: 1.8441 - acc: 0.7224
Epoch 56/100
 - 1s - loss: 1.8443 - acc: 0.7271
Epoch 57/100
 - 1s - loss: 1.8435 - acc: 0.7234
Epoch 58/100
 - 1s - loss: 1.8432 - acc: 0.7200
Epoch 59/100
 - 1s - loss: 1.8438 - acc: 0.7238
Epoch 60/100
 - 1s - loss: 1.8436 - acc: 0.7219
Epoch 61/100
 - 1s - loss: 1.8425 - acc:

 - 1s - loss: 1.8426 - acc: 0.7239
Epoch 87/100
 - 1s - loss: 1.8385 - acc: 0.7235
Epoch 88/100
 - 1s - loss: 1.8424 - acc: 0.7227
Epoch 89/100
 - 1s - loss: 1.8420 - acc: 0.7238
Epoch 90/100
 - 1s - loss: 1.8427 - acc: 0.7209
Epoch 91/100
 - 1s - loss: 1.8415 - acc: 0.7218
Epoch 92/100
 - 1s - loss: 1.8371 - acc: 0.7221
Epoch 93/100
 - 1s - loss: 1.8421 - acc: 0.7239
Epoch 94/100
 - 1s - loss: 1.8420 - acc: 0.7291
Epoch 95/100
 - 1s - loss: 1.8422 - acc: 0.7193
Epoch 96/100
 - 1s - loss: 1.8418 - acc: 0.7209
Epoch 97/100
 - 1s - loss: 1.8399 - acc: 0.7235
Epoch 98/100
 - 1s - loss: 1.8409 - acc: 0.7190
Epoch 99/100
 - 1s - loss: 1.8378 - acc: 0.7264
Epoch 100/100
 - 1s - loss: 1.8402 - acc: 0.7253
4000/4000 [==============================] - 0s 37us/step
[[-0.09601549  0.22609544  0.48284888]
 [-0.29245228  0.2798578  -0.28653333]
 [ 0.15583347 -0.20106408  0.48346913]
 [ 0.08943075 -0.6722016   0.09471997]
 [-0.21997407  0.00940921  0.36042416]
 [ 0.04642984 -0.36303613  0.262235  ]


"\n# neural net\nnet = nn.Sequential(\n    nn.Linear(6, 192),\n    nn.ReLU(),\n    nn.Linear(192, 32),\n    nn.ReLU(),\n    nn.Linear(32, 1)\n)\noptimizer = optim.Adam(net.parameters(), lr=1E-4)\ncriterion = nn.MSELoss()\nnet.train()\n\n# train\nfor epoch in trange(epochs * Nsamples): # using trange instead of range produces a progressbar\n    # randomly select a sample\n    isample = np.random.randint(0, Nsamples)\n    y0 = C(ys[isample])\n \n    # calculate neural net output\n    X = V(Xs[isample]) - torch.cat([origin, origin])\n    z = torch.sum(net(X))\n    # z = torch.sum(1 / torch.norm(X[:,:3], dim=1) / torch.norm(X[:,3:], dim=1) * 1e4)  # this is 100% accurate\n    y = torch.autograd.grad(z, origin, create_graph=True)[0]\n \n    # and perform train step\n    loss = criterion(y,y0)\n    optimizer.zero_grad()   # suggested trick\n    loss.backward()\n    optimizer.step()\n \n    if epoch % 1000 == 0:\n        tqdm.write('%s\t%s\t%s\t%s' %(epoch, N(loss)[0], N(y0)[0], N(y)[0]))\n"

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
8.350230318000058
GPU (s):
0.1842791589999706
GPU speedup over CPU: 45x
